# 2. Time Series Clustering

* Initially thought I would use 1 RNN based model for making predictions
    * But in order to do this I would have to onehot encode the meter_id's (as ordinal would apply a linear relation between id's which doesn't exist)
        * This would add an extra 3500 features and hence make the solution space sufficiently sparse that it is likely our model would give any sort of reasonable performance
        
* My next thought was that instead I could create a separate RNN based model for each of the meters
    * Issue is the time complexity of doing so: even if each model only takes 1 minute to run (of which it is likely they will take longer) that would be 2.4 days of continuous computation
        * And this would have to be repeated for hyperparameter tuning and changes to the model and methodology
            * Very impractical especially considering time constraint of the competition
            
* My plan is to compromise between the 2 by clustering similar time series patterns and having a separate model for making predictions for each of the similar patterns
    * May still have problems with the size of the onehot encoded variables
    * Will also have problem with missing variables as will soon see
    
* If this proves impractical will instead consider models which can better handle categorical values such as decision tree based methods
    * LGB
    * XGBoost
    * Etc.
    
    
#  Loading the data

# Looking at how many missing values we have

#### Conclusions:
* We now know that the exists no single day where all meters have a reading
* The best we can get is a minimum of 11 missing values per day for the last 28 days of the year
    * This is likely as the battery in the meter has died.
* Before that it goes up exponentially making it impractical to use any more days than the last 28
    
* Perhaps we can make a prediction of what those last 28 days are and now we have 0 missing values for last 28 days
    * and can generate and inspect clusters based upon this?
    
* Questions are:
    * 1. just because they have similar patterns in december, are they likely to have similar patterns year round?
        * eg is this enough data to make a real conclusion on
    * 2. is it a good idea as the predicted values may not be accurate
    * 3. Obviously also will have to inspect and experiment with different amounts of clusters
    * 4. May also still have an issue with the onehot encoding if one cluster has too many assigned meters
        * so may be forced to use a different model
        * could maybe get around this with a keras embedding layer
        
# Predicting missing values

# Clustering the data

# Inspecting data